https://www.science.org/doi/10.1126/science.1127647

https://github.com/L1aoXingyu/pytorch-beginner/blob/9c86be785c7c318a09cf29112dd1f1a58613239b/08-AutoEncoder/conv_autoencoder.py

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import torch
import matplotlib.pyplot as plt
from torch import nn
from pathlib import Path
from torch.utils.data import TensorDataset, DataLoader
from lib.modules import (
    evaluate_loop, 
    pad_for_windowing,
    window_session,
    optimization_loop
)
from lib.models import LinearAutoencoder, ConvAutoencoder
from tqdm import tqdm
import plotly.express as px
from datetime import timedelta
from sklearn.model_selection import train_test_split

In [3]:
WINSIZE = 101
DEVICE = 'cuda:0'
RAW_DIR = Path('/home/musa/datasets/eating_raw/')

In [4]:
recordings = []
for rec_dir in RAW_DIR.iterdir():
    recordings.append(rec_dir)
    print(rec_dir.name)

11-07_20_24_32
11-07_17_43_30
11-08_08_27_30
11-08_07_17_47
11-10_08_54_24
11-07_12_58_43
11-01_20_34_28
10-27_00_21_25
11-07_17_29_01
11-01_20_54_52
11-07_15_03_24
10-27_09_45_42
11-02_19_28_19
10-28_13_18_42
10-27_00_20_15


In [5]:
accelerations = []
for session_dir in recordings:
    accel_file = session_dir / f'acceleration-{session_dir.name}.csv'
    acceleration = pd.read_csv(accel_file,skiprows=1).rename({'x': 'x_acc', 'y': 'y_acc', 'z': 'z_acc'}, axis=1)
    # acceleration = acceleration.dropna()

    acceleration_start_time_seconds = float(pd.read_csv(session_dir / accel_file, nrows=1,header=None).iloc[0,0].split()[-1])/1000
    acceleration.timestamp = ((acceleration.timestamp - acceleration.timestamp[0])*1e-9)+acceleration_start_time_seconds # get timestamp in seconds

    accelerations.append(acceleration)

    print(f'Index: {len(accelerations)-1}, Date: {session_dir.name}, nSamples: {len(acceleration)}, Time Elapsed: {timedelta(seconds=acceleration.timestamp.iloc[-1] - acceleration.timestamp.iloc[0])}')


Index: 0, Date: 11-07_20_24_32, nSamples: 30117, Time Elapsed: 0:04:49.392967
Index: 1, Date: 11-07_17_43_30, nSamples: 1005447, Time Elapsed: 2:41:00.547341
Index: 2, Date: 11-08_08_27_30, nSamples: 5125044, Time Elapsed: 1 day, 6:08:45.221686
Index: 3, Date: 11-08_07_17_47, nSamples: 338215, Time Elapsed: 0:54:09.386096
Index: 4, Date: 11-10_08_54_24, nSamples: 1370732, Time Elapsed: 5:06:32.989927
Index: 5, Date: 11-07_12_58_43, nSamples: 776721, Time Elapsed: 2:04:21.712932
Index: 6, Date: 11-01_20_34_28, nSamples: 127133, Time Elapsed: 0:20:21.474008
Index: 7, Date: 10-27_00_21_25, nSamples: 260457, Time Elapsed: 0:51:35.645162
Index: 8, Date: 11-07_17_29_01, nSamples: 90052, Time Elapsed: 0:14:25.199006
Index: 9, Date: 11-01_20_54_52, nSamples: 4080424, Time Elapsed: 22:11:22.872910
Index: 10, Date: 11-07_15_03_24, nSamples: 696054, Time Elapsed: 1:51:27.930080
Index: 11, Date: 10-27_09_45_42, nSamples: 2854125, Time Elapsed: 7:37:03.002257
Index: 12, Date: 11-02_19_28_19, nSampl

In [ ]:
dim_amt = 5
i = 5
fig = px.line(accelerations[i][::dim_amt], x=accelerations[i].index[::dim_amt], y=['x_acc','y_acc','z_acc'])
fig.show(renderer='browser')

In [6]:
skip_idxs = [2, 12, 13]

train_idxs = [1,4,10]
Xtr = []
Xte = []

for i,accel in enumerate(accelerations):
    if i in skip_idxs:
        continue

    X = torch.Tensor(accel[['x_acc','y_acc','z_acc']].values)
    X = pad_for_windowing(X, WINSIZE)
    X = window_session(X, WINSIZE)
    print(X.shape)

    if i in train_idxs:
        Xte.append(X)
    else:
        Xtr.append(X)

Xtr = torch.cat(Xtr)
Xte = torch.cat(Xte)

trainloader = DataLoader(TensorDataset(Xtr), batch_size=64, shuffle=True)
testloader = DataLoader(TensorDataset(Xte), batch_size=64)

torch.Size([30117, 303])
torch.Size([1005447, 303])
torch.Size([338215, 303])
torch.Size([1370732, 303])
torch.Size([776721, 303])
torch.Size([127133, 303])
torch.Size([260457, 303])
torch.Size([90052, 303])
torch.Size([4080424, 303])
torch.Size([696054, 303])
torch.Size([2854125, 303])
torch.Size([6919, 303])


In [9]:
model = ConvAutoencoder(winsize=WINSIZE).to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=3e-4)
criterion = nn.MSELoss()

In [12]:
losses = []
test_losses = []
pbar = tqdm(range(10))
for epoch in pbar:
    lossi = 0
    test_lossi = 0

    for X in trainloader:
        X = X[0].to(DEVICE)

        # Foward Pass
        logits = model(X)
        loss = criterion(logits, X)

        # Backwards Pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Sum Loss
        lossi += loss.item()
    
    # Test model
    for X in testloader:
        X = X[0].to(DEVICE)
        logits = model(X)
        test_lossi += criterion(logits, X).item()

    losses.append(lossi/len(trainloader))
    test_losses.append(test_lossi/len(testloader))
    pbar.set_description(f'Epoch {epoch}: Train Loss: {losses[-1]:.5}, Test Loss: {test_losses[-1]:.5}')

    plt.plot(losses)
    plt.plot(test_losses)

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [01:13<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
torch.save(model.state_dict(), 'dev/autoencorder.pt')

In [ ]:
model.load_state_dict(torch.load('dev/autoencorder.pt'))

In [ ]:
# Recreate a signal with trained model

acceleration = accelerations[3]

Xte = torch.Tensor(acceleration[['x_acc','y_acc','z_acc']].values)
Xte = pad_for_windowing(Xte, WINSIZE)
Xte = window_session(Xte, WINSIZE)

testloader = DataLoader(TensorDataset(Xte), batch_size=64)

Xpred = []
for X in tqdm(testloader):
    X = X[0].to(DEVICE)
    logits = model(X)
    Xpred.append(logits)

Xpred = torch.cat(Xpred)
Xpred = Xpred.view(-1,3,101)[:,:,50].T # unwindow

acceleration['x_pred'] = Xpred[0].cpu().detach()
acceleration['y_pred'] = Xpred[1].cpu().detach()
acceleration['z_pred'] = Xpred[2].cpu().detach()

In [ ]:
dim_amt = 10
fig = px.line(acceleration[::dim_amt], x=acceleration.index[::dim_amt], y=['x_acc','y_acc','z_acc', 'x_pred', 'y_pred', 'z_pred'])
fig.show(renderer='browser')